In [7]:
import networkx as nx

from bokeh.models import (BoxSelectTool, EdgesAndLinkedNodes, HoverTool,
                          MultiLine, Plot, Range1d, Scatter, TapTool)
from bokeh.palettes import Spectral4
from bokeh.plotting import from_networkx, show, output_notebook

G = nx.karate_club_graph()

plot = Plot(width=400, height=400,
            x_range=Range1d(-1.1,1.1), y_range=Range1d(-1.1,1.1))
plot.title.text = "Graph Interaction Demonstration"

plot.add_tools(HoverTool(tooltips=None), TapTool(), BoxSelectTool())

graph_renderer = from_networkx(G, nx.circular_layout, scale=1, center=(0,0))

graph_renderer.node_renderer.glyph = Scatter(size=15, fill_color=Spectral4[0])
graph_renderer.node_renderer.selection_glyph = Scatter(size=15, fill_color=Spectral4[2])
graph_renderer.node_renderer.hover_glyph = Scatter(size=15, fill_color=Spectral4[1])

graph_renderer.edge_renderer.glyph = MultiLine(line_color="#CCCCCC", line_alpha=0.8, line_width=5)
graph_renderer.edge_renderer.selection_glyph = MultiLine(line_color=Spectral4[2], line_width=5)
graph_renderer.edge_renderer.hover_glyph = MultiLine(line_color=Spectral4[1], line_width=5)

graph_renderer.selection_policy = EdgesAndLinkedNodes()
graph_renderer.inspection_policy = EdgesAndLinkedNodes()

plot.renderers.append(graph_renderer)
output_notebook()
show(plot)


Loading BokehJS ...

In [8]:


# %%
import networkx as nx
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import Plot, Range1d, MultiLine, Circle, Scatter, HoverTool, TapTool, BoxSelectTool
from bokeh.palettes import Spectral4

# Step 1: Create a graph
G = nx.karate_club_graph()

# Step 2: Assign a node attribute `node_type` where some nodes are 'circle' and others are 'square'
node_types = ['circle' if node < 10 else 'square' for node in G.nodes()]
nx.set_node_attributes(G, {node: {'node_type': node_type} for node, node_type in zip(G.nodes(), node_types)})

# Step 3: Create a plot
plot = Plot(width=400, height=400,
            x_range=Range1d(-1.5, 1.5), y_range=Range1d(-1.5, 1.5))
plot.title.text = "Graph Interaction Demonstration"

# Create a Bokeh graph from the NetworkX graph
graph_renderer = from_networkx(G, nx.spring_layout, scale=1, center=(0,0))

# Step 4: Use Scatter to draw nodes with different shapes
graph_renderer.node_renderer.glyph = Scatter(size=15, fill_color=Spectral4[0], marker='node_type')

# Prepare the edge renderer (simple lines)
graph_renderer.edge_renderer.glyph = MultiLine(line_color="#CCCCCC", line_alpha=0.8, line_width=1)

plot.renderers.append(graph_renderer)

# Add hover tool
hover = HoverTool(tooltips=[("Type", "@node_type")])
plot.add_tools(hover, TapTool(), BoxSelectTool())

# Step 5: Show the plot
output_notebook()
show(plot)


Loading BokehJS ...

In [9]:


# %%
import networkx as nx
from bokeh.io import output_notebook, show
from bokeh.models import Plot, Range1d, MultiLine, Circle, Scatter, HoverTool, TapTool, BoxSelectTool, ColumnDataSource, LabelSet, Legend, LegendItem, GraphRenderer
from bokeh.plotting import figure
from bokeh.transform import linear_cmap
#tools
from bokeh.models import WheelZoomTool, ResetTool, PanTool

def graph_func():
    # Step 1: Define the graph
    G = nx.karate_club_graph()

    # Step 2: Define node types, colors, and shapes
    node_details = {
        'type1': {'color': '#1f77b4', 'shape': 'circle'},  # Blue circles
        'type2': {'color': '#ff7f0e', 'shape': 'square'}  # Orange squares
    }
    # Assign types to nodes for demonstration
    for i, node in enumerate(G.nodes()):
        G.nodes[node]['type'] = 'type1' if i % 2 == 0 else 'type2'
        G.nodes[node]['color'] = node_details[G.nodes[node]['type']]['color']
        G.nodes[node]['marker'] = node_details[G.nodes[node]['type']]['shape']

    # Step 3: Create the plot
    plot = Plot(width=550, height=450, x_range=Range1d(-1.1, 1.1), y_range=Range1d(-1.1, 1.1))
    plot.title.text = "Graph Interaction Demonstration"

    # Generate layout for the graph
    graph_layout = nx.kamada_kawai_layout(G, scale=2, center=(0,0))
    graph_renderer = from_networkx(G, graph_layout, scale=1, center=(0,0))

    # Step 4: Customize node renderer
    graph_renderer.node_renderer.glyph = Scatter(size=15, fill_color='color', marker='marker')

    # Step 5: Customize edge renderer
    graph_renderer.edge_renderer.glyph = MultiLine(line_color="#CCCCCC", line_alpha=0.8, line_width=1)

    plot.renderers.append(graph_renderer)
    # Step 6: Prepare the data source for labels
    source = ColumnDataSource(data={
        'x': [graph_layout[node][0] for node in G],
        'y': [graph_layout[node][1] for node in G],
        'index': [str(node) for node in G]
    })

    # Step 7: Add labels
    labels = LabelSet(x='x', y='y', text='index', source=source,
                    x_offset=8, y_offset=0, text_font_size="8pt", text_color="black")
    plot.add_layout(labels)


    # Step 8: Create a legend with correct color and shape representation
    legend_items = []
    for node_type, details in node_details.items():
        item_source = ColumnDataSource(data=dict(x=[0], y=[0], size=[10], color=[details['color']], marker=[details['shape']]))
        item_glyph = Scatter(x='x', y='y', size='size', fill_color='color', marker='marker')
        # Invisible renderer in the main plot, but necessary for the legend
        renderer = plot.add_glyph(item_source, item_glyph, visible=False)
        legend_items.append(LegendItem(label=node_type, renderers=[renderer]))

    legend = Legend(items=legend_items, location=(0, 0))
    plot.add_layout(legend, 'right')
    plot.legend.click_policy="hide"
    # Step 9: Finalize plot tools
    hover = HoverTool(tooltips=[("Type", "@type"), ("Index", "@index")])
    plot.add_tools(hover, TapTool(), BoxSelectTool(), ResetTool(), WheelZoomTool(), PanTool())

    output_notebook()
    return plot


In [10]:
from bokeh.layouts import row, gridplot

# Create two plots
plot1 = graph_func()
plot2 = graph_func()



# Layout using gridplot (similar effect for two plots, more useful for complex layouts)
grid_layout = gridplot([plot1, plot2],ncols=1)

# Show the layout
output_notebook()
show(grid_layout)  # or show(grid_layout) to use t


Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

In [11]:
import networkx as nx
from bokeh.io import show, output_notebook
from bokeh.models import Range1d, Plot, MultiLine, Circle, HoverTool, BoxZoomTool, ResetTool, Slider, CustomJS, ColumnDataSource, LinearColorMapper, ColorBar
from bokeh.plotting import from_networkx
from bokeh.layouts import column
from datetime import datetime, timedelta
import random
import numpy as np

# Create a graph with timestamps on edges
G = nx.karate_club_graph()
start_date = datetime.now()

# Adding a 'timestamp' attribute that counts days from the start_date
for u, v in G.edges():
    G.edges[u, v]['timestamp'] = start_date + timedelta(days=random.randint(0, 10))

# Setup the network layout
positions = nx.spring_layout(G)

# Convert timestamps to milliseconds since epoch for use in JavaScript
edge_attrs = {}
for start_node, end_node, _ in G.edges(data=True):
    edge_attrs[(start_node, end_node)] = {'timestamp': G.edges[start_node, end_node]['timestamp'].timestamp() * 1000}

nx.set_edge_attributes(G, edge_attrs)

plot = Plot(width=400, height=400, x_range=Range1d(-1.1, 1.1), y_range=Range1d(-1.1, 1.1), title="Karate Club Graph Evolution")

# Create a graph renderer and pass the layout along with node and edge renderers
graph_renderer = from_networkx(G, positions, scale=1, center=(0,0))
graph_renderer.node_renderer.glyph = Circle(radius = .015, fill_color='skyblue')
graph_renderer.edge_renderer.glyph = MultiLine(line_alpha='alpha', line_color="gray")

# Extract edge timestamps and alpha from graph for use in the slider callback
edge_data = {
    'start': [x[0] for x in G.edges()],
    'end': [x[1] for x in G.edges()],
    'timestamp': [G.edges[start, end]['timestamp'] for start, end in G.edges()],  # Converted to ms
    'alpha': [0.8] * len(G.edges())  # Default alpha value for edges
}

# Create a ColumnDataSource to enable interactive updating
edge_source = ColumnDataSource(edge_data)
graph_renderer.edge_renderer.data_source = edge_source

# Adding the renderer to the plot
plot.renderers.append(graph_renderer)

# Tools
plot.add_tools(HoverTool(tooltips=None), BoxZoomTool(), ResetTool())

# Slider for controlling the visible time range
min_date = min(edge_data['timestamp'])
max_date = max(edge_data['timestamp'])
slider = Slider(start=min_date, end=max_date, value=min_date, step=1000 * 3600 * 24, title="Select Date")

callback = CustomJS(args=dict(source=edge_source), code="""
    var data = source.data;
    var f = cb_obj.value;
    var alphas = data['alpha'];
    for (var i = 0; i < alphas.length; i++) {
        alphas[i] = data['timestamp'][i] <= f ? 0.8 : 0.1;
    }
    source.change.emit();
""")

slider.js_on_change('value', callback)

# Layout setup
layout = column(slider, plot)

# Show output
output_notebook()
show(layout)


Loading BokehJS ...

In [12]:

# %%
import networkx as nx
from bokeh.io import show, output_notebook
from bokeh.models import (Range1d, Plot, MultiLine, Circle, HoverTool, BoxZoomTool, Button,
                          ResetTool, Slider, CustomJS, ColumnDataSource)
from bokeh.plotting import from_networkx
from bokeh.layouts import column
from datetime import datetime, timedelta
import random

# Create a graph with timestamps on edges
G = nx.karate_club_graph()
start_date = datetime.now()

# Adding a 'timestamp' attribute that counts days from the start_date
for u, v in G.edges():
    G.edges[u, v]['timestamp'] = start_date + timedelta(days=random.randint(0, 10))

# Setup the network layout
positions = nx.spring_layout(G)

# Convert timestamps to milliseconds since epoch for use in JavaScript
edge_attrs = {}
for start_node, end_node, _ in G.edges(data=True):
    edge_attrs[(start_node, end_node)] = {'timestamp': G.edges[start_node, end_node]['timestamp'].timestamp() * 1000}

nx.set_edge_attributes(G, edge_attrs)

plot = Plot(width=400, height=400, x_range=Range1d(-1.1, 1.1), y_range=Range1d(-1.1, 1.1), title="Karate Club Graph Evolution")

# Create a graph renderer and pass the layout along with node and edge renderers
graph_renderer = from_networkx(G, positions, scale=1, center=(0,0))
graph_renderer.node_renderer.glyph = Circle(radius=.015, fill_color='skyblue')
graph_renderer.edge_renderer.glyph = MultiLine(line_alpha='alpha', line_color="gray")

# Create and add node labels
labels = LabelSet(x='x', y='y', text='index', source=ColumnDataSource({'x': [positions[i][0] for i in G.nodes()],
                                                                      'y': [positions[i][1] for i in G.nodes()],
                                                                      'index': [str(i) for i in G.nodes()]}),
                  text_font_size="10pt", text_color="black", background_fill_color = "#aaaaee")
plot.add_layout(labels)
# Extract edge timestamps and alpha from graph for use in the slider callback
edge_data = {
    'start': [x[0] for x in G.edges()],
    'end': [x[1] for x in G.edges()],
    'timestamp': [G.edges[start, end]['timestamp'] for start, end in G.edges()],  # Converted to ms
    'alpha': [0.8] * len(G.edges())  # Default alpha value for edges
}

# Create a ColumnDataSource to enable interactive updating
edge_source = ColumnDataSource(edge_data)
graph_renderer.edge_renderer.data_source = edge_source

# Adding the renderer to the plot
plot.renderers.append(graph_renderer)

# Tools
plot.add_tools(HoverTool(tooltips=[('name','@index')] ), BoxZoomTool(), ResetTool(), WheelZoomTool())

# Slider for controlling the visible time range
min_date = min(edge_data['timestamp'])
max_date = max(edge_data['timestamp'])
slider = Slider(start=min_date, end=max_date, value=min_date, step=1000 * 3600 * 24, title="Select Date: " + datetime.utcfromtimestamp(min_date/1000).strftime('%Y-%m-%d'))

# Update slider title to show current date in 'yyyy-mm-dd' format
callback = CustomJS(args=dict(source=edge_source, slider=slider), code="""
    var data = source.data;
    var f = cb_obj.value;
    var alphas = data['alpha'];
    for (var i = 0; i < alphas.length; i++) {
        alphas[i] = data['timestamp'][i] <= f ? 0.8 : 0.1;
    }
    source.change.emit();
    slider.title = 'Select Date: ' + new Date(f).toISOString().slice(0, 10);
""")

slider.js_on_change('value', callback)

# Button for toggling labels
toggle_button = Button(label="Hide Labels", button_type="success")
toggle_callback = CustomJS(args=dict(labels=labels, button=toggle_button), code="""
    labels.visible = !labels.visible;
    button.label = labels.visible ? 'Hide Labels' : 'Show Labels';
""")
toggle_button.js_on_click(toggle_callback)

# Layout setup
layout = column(row(slider, toggle_button), plot)

# Show output
output_notebook()
show(layout)


C:\Users\nyihm\AppData\Local\Temp\ipykernel_313480\1608680808.py:63: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  slider = Slider(start=min_date, end=max_date, value=min_date, step=1000 * 3600 * 24, title="Select Date: " + datetime.utcfromtimestamp(min_date/1000).strftime('%Y-%m-%d'))


Loading BokehJS ...

In [13]:

import networkx as nx
from bokeh.io import show, output_notebook
from bokeh.models import (Range1d, Plot, MultiLine, Circle, HoverTool, BoxZoomTool,
                          ResetTool, Slider, CustomJS, ColumnDataSource, Button, LabelSet)
from bokeh.plotting import from_networkx
from bokeh.layouts import column, row
from datetime import datetime
import random
import pandas as pd

# Create a graph with timestamps on edges
G = nx.karate_club_graph()
start_date = datetime.now()

# Adding a 'timestamp' attribute that counts days from the start_date
for u, v in G.edges():
    G.edges[u, v]['timestamp'] = start_date + timedelta(days=random.randint(0, 10))

# Setup the network layout
positions = nx.spring_layout(G)

# Convert timestamps to milliseconds since epoch for use in JavaScript
edge_attrs = {}
for start_node, end_node, _ in G.edges(data=True):
    edge_attrs[(start_node, end_node)] = {'timestamp': G.edges[start_node, end_node]['timestamp'].timestamp() * 1000}

nx.set_edge_attributes(G, edge_attrs)

plot = Plot(width=400, height=400, x_range=Range1d(-1.1, 1.1), y_range=Range1d(-1.1, 1.1), title="Karate Club Graph Evolution")

# Create a graph renderer and pass the layout along with node and edge renderers
graph_renderer = from_networkx(G, positions, scale=1, center=(0,0))
graph_renderer.node_renderer.glyph = Circle(radius=.015, fill_color='skyblue')
graph_renderer.edge_renderer.glyph = MultiLine(line_alpha='alpha', line_color="gray")

# Create and add node labels
labels = LabelSet(x='x', y='y', text='index', source=ColumnDataSource({'x': [positions[i][0] for i in G.nodes()],
                                                                      'y': [positions[i][1] for i in G.nodes()],
                                                                      'index': [str(i) for i in G.nodes()]}),
                  text_font_size="10pt", text_color="black", background_fill_color = "#aaaaee")
plot.add_layout(labels)
# Extract edge timestamps and alpha from graph for use in the slider callback
edge_data = {
    'start': [x[0] for x in G.edges()],
    'end': [x[1] for x in G.edges()],
    'timestamp': [G.edges[start, end]['timestamp'] for start, end in G.edges()],  # Converted to ms
    'alpha': [0.8] * len(G.edges())  # Default alpha value for edges
}


# Create a ColumnDataSource to enable interactive updating
edge_source = ColumnDataSource(edge_data)
graph_renderer.edge_renderer.data_source = edge_source

# Adding the renderer to the plot
plot.renderers.append(graph_renderer)

# Tools
plot.add_tools(HoverTool(tooltips=None), BoxZoomTool(), ResetTool())

# Slider for controlling the visible time range
min_date = min(edge_data['timestamp'])
max_date = max(edge_data['timestamp'])
time_slider = Slider(start=min_date, end=max_date, value=min_date, step=1000 * 3600 * 24,
                title="Select Date: " + datetime.utcfromtimestamp(min_date / 1000).strftime('%Y-%m-%d'))

# Update slider title to show current date in 'yyyy-mm-dd' format with CustomJS
time_callback = CustomJS(args=dict(source=edge_source, slider=time_slider), code="""
    var data = source.data;
    var f = cb_obj.value;
    var alphas = data['alpha'];
    for (var i = 0; i < alphas.length; i++) {
        alphas[i] = data['timestamp'][i] <= f ? 0.8 : 0.1;
    }
    source.change.emit();
    slider.title = 'Select Date: ' + new Date(f).toISOString().slice(0, 10);
""")

time_slider.js_on_change('value', time_callback)

# Button for toggling labels
toggle_button = Button(label="Hide Labels", button_type="success")
toggle_callback = CustomJS(args=dict(labels=labels, button=toggle_button), code="""
    labels.visible = !labels.visible;
    button.label = labels.visible ? 'Hide Labels' : 'Show Labels';
""")
toggle_button.js_on_click(toggle_callback)

# Slider for node sizes using js_link method
size_slider = Slider(start=.015, end=.75, value=.025, step=.001, title="Node Size")
size_slider.js_link('value', graph_renderer.node_renderer.glyph, 'radius')

# Layout setup
layout = column(row(time_slider, toggle_button),size_slider, plot)

# Show output
output_notebook()
show(layout)



C:\Users\nyihm\AppData\Local\Temp\ipykernel_313480\3937602881.py:65: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  title="Select Date: " + datetime.utcfromtimestamp(min_date / 1000).strftime('%Y-%m-%d'))


Loading BokehJS ...

In [14]:

import networkx as nx
from bokeh.io import show, output_notebook
from bokeh.models import (Plot, Range1d, MultiLine, Scatter, HoverTool, TapTool, BoxSelectTool,
                          ResetTool, WheelZoomTool, PanTool, Legend, LegendItem, ColumnDataSource, LabelSet)
from bokeh.plotting import from_networkx
from bokeh.palettes import Category10

def graph_func():
    # Step 1: Define the graph
    G = nx.karate_club_graph()

    # Step 2: Define node types, colors, and markers
    node_details = {
        'type1': {'color': Category10[10][0], 'marker': 'circle'},  # Blue circles
        'type2': {'color': Category10[10][1], 'marker': 'square'}   # Orange squares
    }

    # Assign attributes to nodes for demonstration
    for i, node in enumerate(G.nodes()):
        G.nodes[node]['type'] = 'type1' if i % 2 == 0 else 'type2'
        G.nodes[node]['color'] = node_details[G.nodes[node]['type']]['color']
        G.nodes[node]['marker'] = node_details[G.nodes[node]['type']]['marker']
    # Generate layout for the graph
    graph_layout = nx.spring_layout(G, scale=2, center=(0,0))


    # Step 3: Create the plot
    plot = Plot(width=550, height=450, x_range=Range1d(-1.1, 1.1), y_range=Range1d(-1.1, 1.1), title="Graph Interaction Demonstration")

    # Step 4: Add edges before nodes
    edge_source = ColumnDataSource(data=dict(
        xs=[[graph_layout[start][0], graph_layout[end][0]] for start, end in G.edges()],
        ys=[[graph_layout[start][1], graph_layout[end][1]] for start, end in G.edges()]
    ))
    plot.add_glyph(edge_source, MultiLine(xs='xs', ys='ys', line_color="#CCCCCC"))

    # Step 5: Separate data sources for node types and create scatter renderers
    renderers = {}
    legend_items = []
    for node_type, details in node_details.items():
        filtered_nodes = [node for node in G.nodes() if G.nodes[node]['type'] == node_type]
        node_source = ColumnDataSource(data={
            'x': [graph_layout[node][0] for node in filtered_nodes],
            'y': [graph_layout[node][1] for node in filtered_nodes],
            'index': [str(node) for node in filtered_nodes],
            'color': [details['color']] * len(filtered_nodes),
            'marker': [details['marker']] * len(filtered_nodes)
        })
        
        # Create scatter renderers for each type
        renderer = plot.add_glyph(node_source, Scatter(x='x', y='y', size=15, fill_color='color', marker='marker'))
        renderers[node_type] = renderer
        
        # Create legend items for each type
        legend_items.append(LegendItem(label=node_type, renderers=[renderer]))

    # Step 6: Create a legend with clickable items that toggle visibility
    legend = Legend(items=legend_items)
    legend.click_policy="hide"  # Allows toggling visibility of nodes per type
    plot.add_layout(legend, 'right')

    # Step 7: Finalize plot tools
    plot.add_tools(HoverTool(tooltips=[("Index", "@index")]), TapTool(), BoxSelectTool(), ResetTool(), WheelZoomTool(), PanTool())

    output_notebook()
    return plot

# Show the plot
show(graph_func())



Loading BokehJS ...

In [15]:

import networkx as nx
from bokeh.io import show, output_notebook
from bokeh.models import (Plot, Range1d, MultiLine, Scatter, HoverTool, BoxZoomTool, ResetTool, WheelZoomTool, 
                          PanTool, Legend, LegendItem, ColumnDataSource, LabelSet, Slider, CustomJS, Button)
from bokeh.palettes import Category10
from bokeh.layouts import column, row
from datetime import datetime, timedelta
import random

def create_combined_graph():
    # Create a graph with additional attributes
    G = nx.karate_club_graph()
    start_date = datetime.now()

    # Assign attributes for circle/square types, colors, and timestamps
    node_details = {
        'type1': {'color': Category10[10][0], 'marker': 'circle'},  # Blue circles
        'type2': {'color': Category10[10][1], 'marker': 'square'}   # Orange squares
    }
    for i, node in enumerate(G.nodes()):
        G.nodes[node]['type'] = 'type1' if i % 2 == 0 else 'type2'
        G.nodes[node]['color'] = node_details[G.nodes[node]['type']]['color']
        G.nodes[node]['marker'] = node_details[G.nodes[node]['type']]['marker']
    
    # Adding a 'timestamp' attribute that counts days from the start_date for edges
    for u, v in G.edges():
        G.edges[u, v]['timestamp'] = (start_date + timedelta(days=random.randint(0, 10))).timestamp() * 1000

    # Network layout
    positions = nx.spring_layout(G, scale=2, center=(0,0))

    # Setup plot
    plot = Plot(width=800, height=600, x_range=Range1d(-1.1, 1.1), y_range=Range1d(-1.1, 1.1), title="Karate Club Graph Evolution")
    plot.add_tools(HoverTool(tooltips=None), BoxZoomTool(), WheelZoomTool(), PanTool(), ResetTool())

    # Draw edges with timestamps
    edge_attrs = {
        'start': [x[0] for x in G.edges()],
        'end': [x[1] for x in G.edges()],
        'xs': [[positions[start][0], positions[end][0]] for start, end in G.edges()],
        'ys': [[positions[start][1], positions[end][1]] for start, end in G.edges()],
        'timestamp': [G.edges[start, end]['timestamp'] for start, end in G.edges()],  # Converted to ms
        'alpha': [0.8] * len(G.edges())  # Default alpha value for edges
    }
    edge_source = ColumnDataSource(edge_attrs)
    plot.add_glyph(edge_source, MultiLine(xs='xs', ys='ys', line_color="gray", line_alpha='alpha'))

    # Draw nodes with different shapes and handle labels for each type
    renderers = {}
    legend_items = []
    label_sets = []  # To keep track of all LabelSets for the toggle functionality

    for node_type, details in node_details.items():
        filtered_nodes = [node for node in G.nodes() if G.nodes[node]['type'] == node_type]
        node_source = ColumnDataSource(data={
            'x': [positions[node][0] for node in filtered_nodes],
            'y': [positions[node][1] for node in filtered_nodes],
            'index': [str(node) for node in filtered_nodes],
            'color': [details['color']] * len(filtered_nodes),
            'marker': [details['marker']] * len(filtered_nodes),
            'size': [10] * len(filtered_nodes)  # Default size
        })
        renderer = plot.add_glyph(node_source, Scatter(x='x', y='y', size='size', fill_color='color', marker='marker', line_color='color'))
        renderers[node_type] = renderer
        legend_items.append(LegendItem(label=node_type, renderers=[renderer]))

        # Create labels for each node type
        labels = LabelSet(x='x', y='y', text='index', source=node_source, text_font_size="8pt", text_color="black")
        plot.add_layout(labels)
        label_sets.append(labels)  # Add to list for later reference in toggle

    # Legend and interactivity
    legend = Legend(items=legend_items)
    legend.click_policy = "hide"  # Toggle visibility per node type
    plot.add_layout(legend, 'right')

    # Slider for timestamp filtering
    min_date = min(edge_attrs['timestamp'])
    max_date = max(edge_attrs['timestamp'])
    time_slider = Slider(start=min_date, end=max_date, value=min_date, step=1000 * 3600 * 24, title="Select Date: " + datetime.utcfromtimestamp(min_date / 1000).strftime('%Y-%m-%d'))
    time_callback = CustomJS(args=dict(source=edge_source, slider=time_slider), code="""
        var data = source.data;
        var f = cb_obj.value;
        var alphas = data['alpha'];
        for (var i = 0; i < alphas.length; i++) {
            alphas[i] = data['timestamp'][i] <= f ? 0.8 : 0.1;
        }
        source.change.emit();
        slider.title = 'Select Date: ' + new Date(f).toISOString().slice(0, 10);
    """)
    time_slider.js_on_change('value', time_callback)

    # Node size slider
    size_slider = Slider(start=5, end=20, value=10, step=1, title="Node Size")
    for renderer in renderers.values():
        size_slider.js_link('value', renderer.glyph, 'size')

    # Button to toggle labels visibility
    toggle_button = Button(label="Hide Labels", button_type="success")
    toggle_callback = CustomJS(args=dict(labels=label_sets), code="""
        labels.forEach(function(label) {
            label.visible = !label.visible;
        });
        cb_obj.label = labels[0].visible ? 'Hide Labels' : 'Show Labels';
    """)
    toggle_button.js_on_click(toggle_callback)

    # Layout configuration
    layout = column(row(plot, column(time_slider, size_slider, toggle_button)))

    output_notebook()
    show(layout)

create_combined_graph()

# %%


C:\Users\nyihm\AppData\Local\Temp\ipykernel_313480\1476880974.py:80: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  time_slider = Slider(start=min_date, end=max_date, value=min_date, step=1000 * 3600 * 24, title="Select Date: " + datetime.utcfromtimestamp(min_date / 1000).strftime('%Y-%m-%d'))


Loading BokehJS ...